I am to predict the destiny of those how were on Titanic.

So to say Road map:

* To know what date I have
* Find correlated features, plot the dependences
* Build new features if necessary
* Apply logistic regression with polynom, find the best suited
* Apply SVM with polynom kernel
* Chose the best model, apply on test data and submit the result.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import fmin_tnc
from sklearn.linear_model import LogisticRegression 
from sklearn import svm
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

## Part 1

In [283]:
df = pd.read_csv('/Users/apple/Documents/!data analysis/Kaggle/train.csv')
df_submit = pd.read_csv('/Users/apple/Documents/!data analysis/Kaggle/test.csv')

In [3]:
df.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [4]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In my opinion, to build the model I don't need Port of Embarkation Cabin number (here we see NaNs) and ticket number.

In [6]:
df = df.iloc[:,[0,1,2,4,5,6,7,9,11]]

In [285]:
#What's more I'm going to make Sex numeric too.
le = LabelEncoder()
le.fit(df["Sex"])
new_sex = le.transform(df["Sex"])
df.insert(2,"Sex_",new_sex)
df = df.drop(["Sex"], axis=1)

In [286]:
df.head()

,PassengerId,Survived,Sex_,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,1,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,0,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S
4,5,0,1,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S


In [10]:
#from SibSp and Parch let's compute a total number of family members on board
sib_arr = df["SibSp"].to_numpy()
parch_arr = df["Parch"].to_numpy()
num = np.array([lambda x, y: x+y, sib_arr, parch_arr])

In [11]:
num = num[1]

In [12]:
df.insert(5,"FamMemb", num)

In [13]:
df.head()

,PassengerId,Survived,Pclass,Sex_num,Sex,FamMemb,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,male,1,22.0,1,0,7.2500,S
1,2,1,1,2,female,1,38.0,1,0,71.2833,C
2,3,1,3,2,female,0,26.0,0,0,7.9250,S
3,4,1,1,2,female,1,35.0,1,0,53.1000,S
4,5,0,3,1,male,0,35.0,0,0,8.0500,S


In [14]:
df_new = df.iloc[:,[0,1,2,3,5,6,9]]
#Age has missing values, we can replace with mean of Age
#Since there are missing data only in Age col, I'm not afraid to replace Nones in others col
df = df.fillna(np.mean(df["Age"]))

In [15]:
df_new.head(1)

,PassengerId,Survived,Pclass,Sex_num,FamMemb,Age,Fare
0,1,0,3,1,1,22.0,7.25


In [16]:
df_new.describe()

,PassengerId,Survived,Pclass,Sex_num,FamMemb,Age,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,1.352413,0.523008,29.699118,32.204208
std,257.353842,0.486592,0.836071,0.477990,1.102743,13.002015,49.693429
min,1.000000,0.000000,1.000000,1.000000,0.000000,0.420000,0.000000
25%,223.500000,0.000000,2.000000,1.000000,0.000000,22.000000,7.910400
50%,446.000000,0.000000,3.000000,1.000000,0.000000,29.699118,14.454200
75%,668.500000,1.000000,3.000000,2.000000,1.000000,35.000000,31.000000
max,891.000000,1.000000,3.000000,2.000000,8.000000,80.000000,512.329200


In [17]:
df_new.describe()

,PassengerId,Survived,Pclass,Sex_num,FamMemb,Age,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,1.352413,0.523008,29.699118,32.204208
std,257.353842,0.486592,0.836071,0.477990,1.102743,13.002015,49.693429
min,1.000000,0.000000,1.000000,1.000000,0.000000,0.420000,0.000000
25%,223.500000,0.000000,2.000000,1.000000,0.000000,22.000000,7.910400
50%,446.000000,0.000000,3.000000,1.000000,0.000000,29.699118,14.454200
75%,668.500000,1.000000,3.000000,2.000000,1.000000,35.000000,31.000000
max,891.000000,1.000000,3.000000,2.000000,8.000000,80.000000,512.329200


Let's look at correlation

In [18]:
df_new.corr(method="pearson")

,PassengerId,Survived,Pclass,Sex_num,FamMemb,Age,Fare
PassengerId,1.000000,-0.005007,-0.035144,-0.042939,-0.057527,0.033207,0.012658
Survived,-0.005007,1.000000,-0.338481,0.543351,-0.035322,-0.069809,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.131900,0.083081,-0.331339,-0.549500
Sex_num,-0.042939,0.543351,-0.131900,1.000000,0.114631,-0.084153,0.182333
FamMemb,-0.057527,-0.035322,0.083081,0.114631,1.000000,-0.232625,0.159651
Age,0.033207,-0.069809,-0.331339,-0.084153,-0.232625,1.000000,0.091566
Fare,0.012658,0.257307,-0.549500,0.182333,0.159651,0.091566,1.000000


Only sex has relativly high correlation with survived parameters, however for model let's try all

## Logistic Regression

In [19]:
dataX = df_new.iloc[:,[2,3,4,5,6]]
dataY = df_new.iloc[:,1]

In [20]:
dataX

,Pclass,Sex_num,FamMemb,Age,Fare
0,3,1,1,22.000000,7.2500
1,1,2,1,38.000000,71.2833
2,3,2,0,26.000000,7.9250
3,1,2,1,35.000000,53.1000
4,3,1,0,35.000000,8.0500
...,...,...,...,...,...
886,2,1,0,27.000000,13.0000
887,1,2,0,19.000000,30.0000
888,3,2,1,29.699118,23.4500
889,1,1,0,26.000000,30.0000


In [21]:
sc = StandardScaler()
dataX = sc.fit_transform(dataX)

/Users/apple/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/apple/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [22]:
X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size = 0.3, random_state = 42)

In [23]:
X_train = np.c_[np.ones(len(X_train)),X_train]
X_test = np.c_[np.ones(len(X_test)),X_test]
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [24]:
y_test

array([1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 1])

In [25]:
model1 = LogisticRegression(solver="lbfgs")
model1.fit(X_train, y_train)
predicted = model1.predict(X_train)
prediction_test = model1.predict(X_test)
parameters1 = model1.coef_

In [26]:
parameters1 = parameters1.reshape(-1,1)

In [27]:
#accuracy train and test
acc_train1 = metrics.accuracy_score(y_train, predicted)
acc_test1 = metrics.accuracy_score(y_test, prediction_test)
print(acc_train1, acc_test1)

0.797752808988764 0.8022388059701493


In [28]:
#F1 score (precision and recall) for test and train
f1_train1 = metrics.f1_score(y_train, predicted)
f1_test1 = metrics.f1_score(y_test, prediction_test)
print(f1_train1, f1_test1)

0.7123287671232876 0.7488151658767773


In [29]:
#Let's make dict with our results
accuracy_dict = {"acc_train":[acc_train1],
                "acc_test":[acc_test1],
                "f1_train":[f1_train1],
                "f1_test":[f1_test1]}

In [30]:
#let's make function,that computes and adds accuracy to our dict
def add_accuracy(y_train, y_test, predict_train, predicted_test):
    acc_train = metrics.accuracy_score(y_train, predicted)
    acc_test = metrics.accuracy_score(y_test, prediction_test)
    f1_train = metrics.f1_score(y_train, predict_train)
    f1_test = metrics.f1_score(y_test, predicted_test)
    accuracy_dict["acc_train"].append(acc_train)
    accuracy_dict["acc_test"].append(acc_test)
    accuracy_dict["f1_train"].append(f1_train)
    accuracy_dict["f1_test"].append(f1_test)
    return acc_train, acc_test, f1_train, f1_test

Logistic Regression with polynom features

In [31]:
#to find out the best polynom degree I'll try to apply it with different degrees(hope, 2,3,4 would be enough)
def poly_reg(degree, X_train, X_test, y_train, y_test, model):
    poly = PolynomialFeatures(degree = degree, interaction_only=False, include_bias=True)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.fit_transform(X_test)
    
    model.fit(X_train_poly, y_train)
    predict_train = model.predict(X_train_poly)
    predicted_test = model.predict(X_test_poly)
    parameters = model.coef_
    iter_ = model.n_iter_
    
    return iter_, predict_train, predicted_test, parameters

In [32]:
model2 = LogisticRegression(solver="sag", penalty = "l2", max_iter=1200)
iter_, predict_train2, predicted_test2, parameters2= poly_reg(2, X_train, X_test, y_train, y_test, model2)

In [33]:
#the number of iterations required for converge
iter_

array([1144], dtype=int32)

In [34]:
add_accuracy(y_train, y_test, predict_train2, predicted_test2)

(0.797752808988764, 0.8022388059701493, 0.7488151658767772, 0.7729468599033817)

I've tried degree 3 and havn't converged even with 1500 iterations. In my opinion for these data it's big enough, that's why I'll try another model

In [35]:
model3 = svm.SVC(kernel = "poly", gamma="scale")
model3.fit(X_train, y_train)
prediction_train3 = model3.predict(X_train)
prediction_test3 = model3.predict(X_test)
parameters3 = model3.dual_coef_
add_accuracy(y_train, y_test, prediction_train3, prediction_test3)

(0.797752808988764, 0.8022388059701493, 0.75, 0.743718592964824)

In [36]:
accuracy_dict

{'acc_train': [0.797752808988764, 0.797752808988764, 0.797752808988764],
 'acc_test': [0.8022388059701493, 0.8022388059701493, 0.8022388059701493],
 'f1_train': [0.7123287671232876, 0.7488151658767772, 0.75],
 'f1_test': [0.7488151658767773, 0.7729468599033817, 0.743718592964824]}

### Predictions for scoring Part1


In [37]:
df_submit.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S


In [38]:
df_submit = df_submit.fillna(np.mean(df["Age"]))

In [39]:
df_submit.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,29.6991,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,29.6991,S


In [40]:
df_submit = df_submit.iloc[:,[0,1,3,4,5,6,8]]

In [41]:
sexlst = [1 if i=="male" else 2 for i in df_submit["Sex"]]

In [42]:
df_submit.insert(3,"Sex_num",sexlst)

In [43]:
df_submit = df_submit.drop(["Sex"],axis=1)

In [44]:
sib_arr = df_submit["SibSp"].to_numpy()
parch_arr = df_submit["Parch"].to_numpy()
num = np.array([lambda x, y: x+y, sib_arr, parch_arr])
num = num[1]
df_submit.insert(5,"FamMemb", num)

In [45]:
df_submit.head()

,PassengerId,Pclass,Sex_num,Age,SibSp,FamMemb,Parch,Fare
0,892,3,1,34.5,0,0,0,7.8292
1,893,3,2,47.0,1,1,0,7.0000
2,894,2,1,62.0,0,0,0,9.6875
3,895,3,1,27.0,0,0,0,8.6625
4,896,3,2,22.0,1,1,1,12.2875


In [46]:
dataX = df_submit.drop(columns=["PassengerId","SibSp","Parch" ])

In [47]:
dataX.head(1)

,Pclass,Sex_num,Age,FamMemb,Fare
0,3,1,34.5,0,7.8292


In [48]:
dataX = sc.fit_transform(dataX)

/Users/apple/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/apple/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [49]:
dataX = np.c_[np.ones(len(dataX)), dataX]

In [50]:
poly = PolynomialFeatures(2, interaction_only=False, include_bias=True)
dataX_poly = poly.fit_transform(dataX)

In [51]:
result = model2.predict(dataX_poly)

In [52]:
result

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [53]:
result = np.c_[df_submit.iloc[:,0],result]

In [54]:
result_df = pd.DataFrame(data=result, columns=["PassengerId", "Survived"])
print(result_df)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


In [55]:
#np.savetxt("result.csv", result, delimiter=",")
result_df.to_csv('result.csv', index=False)

In [56]:
result_svm = model3.predict(dataX)

In [58]:
result_svm = np.c_[df_submit.iloc[:,0],result_svm]

result_df_svm = pd.DataFrame(data=result_svm, columns=["PassengerId", "Survived"])
print(result_df_svm)

result_df_svm.to_csv('result_svm.csv', index=False)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


## Results

**SVM model** gives higher score when submitting - **0.77511** against **Poly Log Reg** - **0.72248**

Transforming all variables to categoric didn't appear to get higher scores, I should remaster it.